### ECDSA-like Signature Scheme

The server implements an ECDSA-like signature scheme which is summarised below.

With public parameters $(p, E(\mathbb{F}_p, a, b), G, n, H)$ ($n$ is the curve order, $H$ is the hash function)

**Key Generation:** The signer chooses a private key $d \in [1, n-1]$ and computes their public key $Q = dG$ 

**Signing a message $m$**:
1. Calculate $h = H(m)$
2. Select random $k_1, k_2 \in [1, n-1]$
3. Calculate $r_1 = (k_1 G)_x$, $r_2 = (k_2 G)_y$. If $r_1 = r_2 = 0$, go back to step 2
4. Calculate $s \equiv k^{-1} (hr_1 - r_2d) \pmod n$
5. The signature is $(r_1, r_2, s)$

**Verifying a message $m$ and signature $(r_1, r_2, s)$**:
1. Ensure $r_1, r_2, s \in [1, n-1]$. Return invalid if not
2. Calculate $h = H(m)$
3. Calculate $v_1 = hr_1 s^{-1}$ and $v_2 = r_2 s^{-1}$
4. Calculate $(x_1, y_1) = v_1 G - v_2 Q$
5. The signature is valid if $x_1 \equiv r_1 \pmod n$.

The signature scheme itself is probably secure, but an implementation flaw allows us to recover the private key given a few signatures. Specifically, the nonce $k_1$ is biased and all nonces have only 340 bits of randomness, with a shared suffix. This kind of attack has been covered in the literature (https://eprint.iacr.org/2019/023.pdf). The attack works by picturing the situation as a hidden number problem instance, which can be solved using lattice-based techniques.

### Hidden Number Problem

A (simplified) version of the hidden number problem can be stated as follows.

Let $p$ be a prime and let $d \in \mathbb{F}_p$ be an unknown integer. Recover $d$ given pairs of integers $\{ (t_i, a_i) \}_{i=1}^m$ such that
$$k_i - t_i d - a_i \equiv 0 \pmod p$$
and $|k_i| < B$ for some $B < p$. The $k_i$ are unknown.

This problem can be easily solved given a sufficient number of pairs and small enough $B$.

Consider the lattice generated by the rows of the matrix

$$M = \begin{bmatrix} p & 0 & 0 & \cdots & 0 & 0 \\ 0 & p & 0 & \cdots & 0 & 0 \\ \vdots &  & \ddots & & & \vdots \\ 0 & 0 & \cdots & p & 0 & 0 \\ t_1 & t_2 & \cdots & t_m & B/p & 0 \\ a_1 & a_2 & \cdots & a_m & 0 & B \end{bmatrix}$$

We'll call the first $m$ rows of this matrix $p$-rows. Notice that if we multiply the second-to-last row by $d$, add the last row to it, and subtract appropriate multiples of the $p$-rows, we get the vector

$$v_k = (k_1, k_2, \ldots, k_m, Bd/p, B)$$

which is a short vector since the $k_i$ are small, so can be recovered by solving SVP!

### Finding the HNP Instance

Our goal is to turn the signatures from the service into an instance of the HNP. It should be quite obvious that the hidden number is the private key $d$, and the unknown $k_i$'s are the biased nonces. In fact, if we look at how the verification algorithm works, we see that it almost tells us how to write the equations. Let 

$$t_i = -2^{-25}r_{i2} s_i^{-1} \qquad a_i = 2^{-25}h_ir_{i1} s_i^{-1}$$

Each nonce has `340` bits of randomness, plus `25` bits that we don't know (but is a common suffix for all nonces). It turns out that just knowing that the top `19` bits are all 0 isn't quite enough to recover the private key as we only have a limited number of samples. We'll need to optimise a bit. Suppose we are given $m$ signatures by the server. What we can do is eliminate the unknown common suffix by taking the $m-1$ points of data given by $\{t_i' = t_0 - t_i\}_{i=1}^m$ and $\{a_i' = a_0 - a_i\}_{i=1}^m$ as our inputs to the hidden number problem.

Write the nonce $k_i$ as $k_i = p_i 2^{25} + c$ where $p_i$ is the `340` random bits, and $c$ is the unknown common suffix.

To help make the algebra a bit less messy, let $b_i = (h_i r_{i1} - r_{i2}d)^{-1}$.

Then,

$$\begin{aligned} t_i' &\equiv t_0 - t_i \\ &\equiv -2^{-25} (r_{02} k_0 b_0 - r_{i2} k_i b_i) \pmod n \end{aligned}$$

and

$$\begin{aligned} a_i' &\equiv a_0 - a_i \\ &\equiv 2^{-25} (h_0 r_{01} k_0 b_0 - h_i r_{i1} k_i b_i) \pmod n \end{aligned}$$

So,

$$\begin{aligned} t_i' d + a_i' &\equiv 2^{-25}(-d(r_{02} k_0 b_0 - r_{i2} k_i b_i) + h_0 r_{01} k_0 b_0 - h_i r_{i1} k_i b_i) \\ &\equiv 2^{-25}(h_0 r_{01} k_0 b_0 - r_{02} k_0 b_0 d -(h_i r_{i1} k_i b_i - r_{i2} k_i b_i d)) \\ &\equiv 2^{-25}(k_0 b_0 (h_0 r_{01} - r_{02} d) - k_i b_i (h_i r_{i1} - r_{i2} d)) \\ &\equiv 2^{-25} (k_0 b_0 b_0^{-1} - k_i b_i b_i^{-1}) \\ &\equiv 2^{-25} (k_0 - k_i) \\ &\equiv 2^{-25} (p_0 2^{25} + c - (p_i 2^{25} + c)) \\ &\equiv p_0 - p_i \pmod n \end{aligned}$$

Therefore...

$$(p_0 - p_i) - t_i' d - a_i' \equiv 0 \pmod n$$

and this is precisely the hidden number problem setting described above! Even better, it turns out that we have enough signatures to recover the private key with this optimisation.

All that's left is to throw the matrix at LLL, recover the private key, sign the auth message and grab the flag!

Solve script:

```python
import os
os.environ['PWNLIB_NOTERM'] = 'True'
from pwn import remote, process
import ecdsa
import hashlib

Curve = ecdsa.NIST384p
G = Curve.generator
n = Curve.order

def connect():
    return remote('0.0.0.0', 1337, level='error')
    # return process('../challenge/server.py')

def recv(conn):
    o = conn.recvline().decode()
    print('[<]', o)
    return o

def send(conn, d):
    print('[>]', d)
    conn.sendline(d)

def read_menu(conn):
    [recv(conn) for _ in range(5)]

def get_sig(conn, msg):
    read_menu(conn)
    send(conn, 's')
    recv(conn)
    send(conn, msg.hex())
    sig = [int(x) for x in recv(conn).split(' ')]
    return sig

def verify(conn, msg, sig):
    read_menu(conn)
    send(conn, 'v')
    recv(conn)
    send(conn, msg.hex())
    recv(conn)
    send(conn, ' '.join(str(x) for x in sig))
    return recv(conn)

def recover_privkey(n, hashes, sigs, B, lo):
    Zn = Zmod(n)
    N = len(sigs)-1
    M = [[0]*i + [n] + [0]*(N-i-1+2) for i in range(N)]

    Ts = [int(Zn(-r2/((2^lo)*Zn(s)))) for r1, r2, s in sigs]
    M.append([int(Zn(Ts[0] - t)) for t in Ts[1:]] + [B/n, 0])

    As = [int(Zn(h*r1/((2^lo)*Zn(s)))) for h, (r1, r2, s) in zip(hashes, sigs)]
    M.append([int(Zn(As[0] - a)) for a in As[1:]] + [0, B])

    M = Matrix(M)
    M = M.LLL()

    for row in M:
        if row[-1] == B:
            return Zn(row[-2]*n/B)

conn = connect()
recv(conn)

hashes = []
sigs = []
for m in range(10):
    msg = os.urandom(6)
    hashes.append(int(hashlib.sha384(msg).hexdigest(), 16))
    sigs.append(get_sig(conn, msg))

d = recover_privkey(int(n), hashes, sigs, 2^340, 25)
print('[+] recovered d:', d)

auth_msg = b'I know alll of your secrets!'
h = int(hashlib.sha384(auth_msg).hexdigest(), 16)
k1, k2 = 133, 337
r1 = (k1*G).x()
r2 = (k2*G).y()
s = inverse_mod(k1, n)*(Integer(h*r1) - r2*d) % n
verify(conn, auth_msg, (r1, r2, s))
conn.close()
```